In [1]:
from MOF_build import buildUIO66
from MDprepare import move2C
from MDprepare import ff_gen as ff
from MDprepare import itp_process as ip
import numpy as np

In [4]:
'''
#STEP1 Build a MOF model 
'''

linker_pdb = "../linkers4demo/66.pdb"
x_num, y_num, z_num, x_scalar, y_scalar, z_scalar = 3,3,3,14.5,14.5,14.5
outnode, outlinker, outcut, outall = ("66node", "66linker", "66cut", "66all")  # name of output file
O1_index, O2_index, O3_index = 9,11,10  ##NOTE:O1->O2//short edge O-O ,O1->O3//long edge O-O

In [6]:
mof = buildUIO66.UIO66(linker_pdb, O1_index, O2_index, O3_index, x_num, y_num, z_num,x_scalar, y_scalar, z_scalar )
mof.buildall(outnode, outcut, outlinker,outall)
mof.view(outall)
#move atoms to box center and reset box size 
move2C.atoms2c('66all',0.9,1.6,0.9,5,3.5,4.3) 


points number:  64
linkers number:  252
total atoms: 10104
ZRR   384
OMM   256
OHH   256
CUT   264
UNK   252

Time cost (s):   0:00:38.565303


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [ ]:
'''
#STEP2 generate linker forcefield 
'''


linkerpath='Residues'
basis = 'sto-3g'
charge = -2
grofile_name,residue_name ='cph','UNK'
ff.ff_gen(linkerpath,charge,basis,grofile_name,residue_name)


In [ ]:
"""step3 generate topology file for MD simulation"""


middlelines, sectorname = ip.parsetop("MDdemo/itp/template.top")
res_num = (
    "ZRR            48\nOMM            32\nOHH            32\nCUT             60\n"
    + "UNK            18\nSOL            1700\nK            10\n"
)
ip.top_gen("itps/cph.itp", middlelines, "em.top", res_num)

In [ ]:
''' 
insert 1000 water molecules in the box before MD running
'''
a = 'MDdemo/mdfiles'
mdp = 'MDdemo/mdp'
!gmx insert-molecules -f new66all.gro -ci MDdemo/itp/tip3p.gro -nmol 1700 -o $a/mof_sol.gro -scale 0.4
!gmx insert-molecules -f $a/mof_sol.gro -ci MDdemo/itp/K.gro -nmol 10 -o $a/em.gro  -scale 0.4

In [ ]:
''' run MD simulation ''' 
!gmx grompp -f mdfiles/mdp/em.mdp   -c $a/em.gro    -p $a/em.top   -o $a/em.tpr -maxwarn 1 
!gmx mdrun -s $a/em.tpr -deffnm $a/em_out -g $a/em.log
#!gmx grompp -f $mdp/nvt.mdp -c $a/em_out.gro -p $a/em.top -o $a/nvt.tpr -maxwarn 1 
#!gmx mdrun  -s $a/nvt.tpr  -deffnm $a/nvt_out -g $a/nvt.log
#!gmx grompp  -f $mdp/npt.mdp -c $a/nvt_out.gro -p $a/em.top -o $a/npt.tpr -maxwarn 1 
#!gmx mdrun  -s $a/npt.tpr  -deffnm $a/npt_out -g $a/npt.log
#!gmx grompp -f $mdp/md.mdp -c $a/npt_out.gro  -p $a/em.top -o $a/md.tpr -maxwarn 1 
#!gmx mdrun  -s $a/md.tpr  -deffnm $a/md_out -g $a/md.log

In [ ]:
!gmx grompp -f $mdp/nvt.mdp -c $a/em_out.gro -p $a/em.top -o $a/nvt.tpr -maxwarn 1 
!gmx mdrun  -s $a/nvt.tpr  -deffnm $a/nvt_out -g $a/nvt.log

In [ ]:
!gmx grompp  -f $mdp/npt.mdp -c $a/nvt_out.gro -p $a/em.top -o $a/npt.tpr -maxwarn 1 
!gmx mdrun  -s $a/npt.tpr  -deffnm $a/npt_out -g $a/npt.log